<a href="https://colab.research.google.com/github/onishchenkoar/dungeons-and-dragons-and-probabilities/blob/main/Cleanup_All_Rolls_TalDorei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleanup of _All Rolls - Tal'Dorei_ table

I'm using [*All Rolls - Tal'Dorei* table]((https://docs.google.com/spreadsheets/d/1OEg29XbL_YpO0m5JrLQpOPYTnxVsIg8iP67EYUrtRJg/edit?usp=sharing) for a personal project of mine. I've found some inconsistencies in the document; so here is my attempt to improve it. I have not gone verifying the data row-by-row, but rather employed some consistency checks on each column.

## Couple of notes

### Note 1
In the code, whenever there is a row number (for example, 3942 in `df.loc[3942, 'Time']`), add 2 to it to find a corresponding row in the [original document]((https://docs.google.com/spreadsheets/d/1OEg29XbL_YpO0m5JrLQpOPYTnxVsIg8iP67EYUrtRJg/edit?usp=sharing). This happens because in Pandas indexing starts from 0 and the header is not counted as a row. In Google Spreadsheets, indexing starts from 1 and the header is one of the rows. So, whenever row 3942 (like in the example above) is addressed in the code, it is row 3944 in the Google doc.

### Note 2
There are a number of empty rows (3032, 3324 are notes on missing audio/video; 7243 to 7248 seem to be some artifact of manual processing). I noticed them in the middle of my process and decided to keep them until the very end, so I don't mess the indexing.

### Note 3
I'm structuring the code so that only parts that make changes to the table are executable. I'm formatting all code involving the search for mistakes as Markdown cells. I did it this way, because a) when searching, the output is cumbersome, b) finding typos and inconsistencies involves a lot of manual labor that cannot be captured in code.

## Imports and initialization

In [1]:
import json

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


pd.set_option('display.max_rows', 15000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 500)

df = pd.read_csv("drive/MyDrive/All Rolls - Tal'Dorei - All Episodes.csv",
                 skipinitialspace=True
)

## *Episode* column

One typo.

In [2]:
# Correct typos:
df.loc[858, 'Episode'] = '8' # was 9

The Episode column needs a change of format, so that it can be sorted alphabetically.  

In [3]:
# Format Episode column, so that it is appropriate for sorting.
# Episode '2' becomes 'C1E002'; episode '31 p1' becomes 'C1E031 p1'.
# This is the formatting used in Campaign 2 table of rolls.
df['Episode'] = df['Episode'].astype(str)
df['Episode'] = df['Episode'].map(lambda x: 'C1E' + x.zfill(3
                                                            if len(x) <= 3
                                                            else 6 
                                                           )
                                 )

## *Time* column

### Malformed timestamps

I found malformed timestamps using the following code:
```python
for i, t in df['Time'].items():
  try:
    pd.to_datetime(t, format='%H:%M:%S')
  except:
    print(i, t)

# >>> 3942 1:10
#     8023 2:12;26
#     9282 1:27:43)
```


The code to correct them:

In [4]:
# Correct malformed timestamps:
df.loc[3942, 'Time'] = '1:10:00' # was 1:10
df.loc[8023, 'Time'] = '2:12:26' # was 2:12;26
df.loc[9282, 'Time'] = '1:27:43' # was 1:27:43)

### Typos in the timestamps

I checked whether the timestamps within one episode are always increasing (or same).

```python
temp = df[['Episode', 'Time']].copy().dropna()
temp.loc[:, 'Time'] = pd.to_datetime(temp['Time']).dt.time

next_time_is_higher_or_same = temp.iloc[1:, 1].to_numpy() >= temp.iloc[:-1, 1].to_numpy()
next_ep_is_higher = temp.iloc[1:, 0].to_numpy() > temp.iloc[:-1, 0].to_numpy()
monotonic_row_mask = np.logical_xor(next_time_is_higher_or_same, next_ep_is_higher)
monotonic_row_mask = np.concatenate(([True], monotonic_row_mask))
# I have to augment the mask because I used dropna() when assigning temp
# and rows 3032, 3324, 7246-7249 are empty.
monotonic_row_mask = np.concatenate((monotonic_row_mask[:3031],
                                     [True],
                                     monotonic_row_mask[3031:]
                                    )
                                   )
monotonic_row_mask = np.concatenate((monotonic_row_mask[:3323],
                                     [True],
                                     monotonic_row_mask[3323:]
                                    )
                                   )
monotonic_row_mask = np.concatenate((monotonic_row_mask[:7243],
                                     [True, True, True, True, True],
                                     monotonic_row_mask[7243:]
                                    )
                                   )
idx = np.where(~monotonic_row_mask)[0]

print(idx)
print()
print(len(idx))

# >>>[  336   520   804  1061  1120  1183  1535  1611  1681  1826  1834  2045
#  ...    
#  13331 13333 13335 13346 13376 13412 13430 13505]

# 140
```

In 140 cases, they are not. Sometimes, those are typos, sometimes, those are just rows out of order.

The code that corrects the typos I managed to find:

In [5]:
# Correct mistyped timestamps:
df.loc[[334, 335], "Time"] = "1:55:21" # was 1:56:21
df.loc[520, "Time"] = "2:52:59" # was 2:22:59  
df.loc[803, "Time"] = "3:28:00" # was 3:39:00
df.loc[1120, "Time"] = "1:17:18" # was 1:17:13
df.loc[1183, "Time"] = "0:41:37" # was 0:31:37
df.loc[1534, "Time"] = "2:17:22" # was 2:27:22
df.loc[1610, "Time"] = "0:38:27" # was 0:38:47
df.loc[1680, "Time"] = "2:39:01" # was 2:39:04
df.loc[1826, "Time"] = "2:51:01" # was 2:50:15
df.loc[1834, "Time"] = "2:57:25" # was 2:57:23
df.loc[2518, "Time"] = "3:25:02" # was 3:35:02
df.loc[3526, "Time"] = "1:13:04" # was 1:03:04
df.loc[3571, "Time"] = "1:53:06" # was 1:53:16
df.loc[3611, "Time"] = "0:02:12" # was 0:05:26
df.loc[3622, "Time"] = "0:28:22" # was 0:28:23
df.loc[3824, "Time"] = "1:20:10" # was 1:18:28
df.loc[4127, "Time"] = "2:33:15" # was 2:30:55
df.loc[4277, "Time"] = "0:52:06" # was 0:54:06
df.loc[4494, 'Time'] = '2:52:32' # was 2:53:32
df.loc[4614, "Time"] = "2:19:48" # was 2:19:28
df.loc[4893, "Time"] = "0:52:01" # was 0:51:01
df.loc[5108, "Time"] = "3:05:10" # was 3:01:10
df.loc[5118, "Time"] = "1:09:58" # was 1:00:58
df.loc[5585, "Time"] = "2:38:14" # was 2:28:14
df.loc[5592, "Time"] = "2:42:15" # was 1:42:15
df.loc[5636, "Time"] = "1:34:30" # was 1:32:30
df.loc[[5674, 5675], "Time"] = "2:45:47" # was 2:52:47
df.loc[5707, "Time"] = "1:51:54" # was 1:51:14
df.loc[5905, "Time"] = "1:54:47" # was 1:54:17
df.loc[5936, "Time"] = "3:03:57" # was 3:05:59
df.loc[5937, "Time"] = "3:03:59" # was 3:05:59
df.loc[5941, "Time"] = "3:04:47" # was 3:04:04
df.loc[5956, "Time"] = "3:08:53" # was 2:08:53
df.loc[5975, "Time"] = "3:32:27" # was 3:32:37
df.loc[6022, "Time"] = "3:58:26" # was 2:58:26
df.loc[6193, "Time"] = "1:15:20" # was 1:50:20
df.loc[6325, "Time"] = "1:00:31" # was 1:01:31  
df.loc[[6359, 6360], "Time"] = "1:41:48" # was 1:45:48
df.loc[6492, "Time"] = "2:42:53" # was 3:42:53 
df.loc[6536, "Time"] = "3:28:13" # was 3:18:13
df.loc[[6705, 6706, 6707, 6709, 6710, 6711], "Time"] = "2:42:10" # was 2:40:10
df.loc[6741, "Time"] = "3:15:33" # was 3:14:52
df.loc[6742, "Time"] = "3:15:45" # was 3:05:42
df.loc[7187, "Time"] = "2:54:57" # was 2:54:27
df.loc[7404, "Time"] = "0:37:30" # was 0:27:30
df.loc[7380, "Time"] = "0:11:18" # was 0:11:48
df.loc[7392, "Time"] = "0:27:28" # was 0:27:56
df.loc[[7395, 7396], "Time"] = "0:27:33" # was 0:27:31
df.loc[7410, "Time"] = "0:42:42" # was 0:42:32
df.loc[7412, "Time"] = "0:49:21" # was 0:42:21
df.loc[7506, "Time"] = "3:00:23" # was 2:00:23
df.loc[7521, "Time"] = "3:08:23" # was 3:11:23
df.loc[7535, "Time"] = "3:15:49" # was 3:15:37
df.loc[7547, "Time"] = "3:32:04" # was 3:31:54
df.loc[7560, "Time"] = "3:52:08" # was 3:22:12
df.loc[7959, "Time"] = "1:18:50" # was 1:48:50
df.loc[7960, "Time"] = "1:18:56" # was 1:48:56
df.loc[7991, "Time"] = "1:49:57" # was 1:49:37
df.loc[8087, "Time"] = "3:23:15" # was 3:27:15
df.loc[8247, "Time"] = "0:43:03" # was 0:32:03
df.loc[8379, "Time"] = "3:57:09" # was 2:57:09
df.loc[8397, "Time"] = "4:18:15" # was 4:19:20
df.loc[8470, "Time"] = "2:47:12" # was 2:27:12
df.loc[8472, "Time"] = "2:56:44" # was 2:45:44
df.loc[8892, "Time"] = "2:26:50" # was 2:25:50
df.loc[9239, "Time"] = "0:27:14" # was 2:27:14 
df.loc[9309, "Time"] = "2:22:04" # was 2:02:04
df.loc[9493:9500, "Time"] = "2:46:09" # was 2:42:29
df.loc[[9631, 9632], "Time"] = "0:37:15" # was 0:37:35
df.loc[9694, "Time"] = "1:38:22" # was 1:38:32
df.loc[9751, "Time"] = "2:36:59" # was 2:26:59
df.loc[9776, "Time"] = "3:17:01" # was 3:17:22 
df.loc[9852, "Time"] = "0:27:43" # was 0:26:43
df.loc[9876, "Time"] = "0:47:36" # was 0:43:46
df.loc[10142, "Time"] = "2:31:13" # was 2:26:23
df.loc[10257, "Time"] = "3:54:40" # was 3:54:17
df.loc[10322, "Time"] = "4:41:45" # was 4:01:35
df.loc[10372, "Time"] = "5:18:30" # was 5:18:17
df.loc[10767, "Time"] = "3:03:00" # was 3:02:26
df.loc[10834, "Time"] = "3:54:07" # was 2:54:07
df.loc[10870, "Time"] = "1:07:55" # was 1:32:15
df.loc[10895, "Time"] = "3:34:09" # was 3:24:09
df.loc[11196, "Time"] = "3:59:50" # was 3:59:15
df.loc[11284, "Time"] = "3:05:21" # was 2:05:21
df.loc[11538, "Time"] = "2:29:18" # was 2:29:28 
df.loc[11651, "Time"] = "2:38:02" # was 2:39:02
df.loc[[11698, 11699], "Time"] = "3:29:05" # was 3:28:05
df.loc[11701, "Time"] = "3:30:33" # was 3:20:23
df.loc[11800, "Time"] = "1:12:46" # was 1:21:46
df.loc[11858, "Time"] = "2:16:55" # was 2:15:55
df.loc[11988, "Time"] = "1:25:52" # was 1:24:42
df.loc[12016, "Time"] = "2:25:43" # was 2:25:42
df.loc[12027, "Time"] = "2:32:09" # was 1:32:09
df.loc[12558, "Time"] = "2:51:14" # was 2:21:14
df.loc[12681, "Time"] = "1:46:34" # was 1:45:34
df.loc[13144, "Time"] = "0:16:40" # was 0:16:50
df.loc[13346, "Time"] = "3:35:28" # was 3:25:28
df.loc[13411, "Time"] = "4:17:56" # was 5:17:56
df.loc[13430, "Time"] = "4:38:29" # was 4:38:28
df.loc[13505, "Time"] = "2:13:15" # was 1:13:15 
df.loc[13506, "Time"] = "2:19:35" # was 1:19:35

### Mismatching timestamps for rolls with advantage/disadvantage

I've noticed that, in general, rolls with advantage and disadvantage come in pairs and have the same timestamp, disregarding the actual timing for one of them.

I have found three timestamps violating this rule.

Correction:

In [6]:
# Replace timestamps that do not match their advantage/disadvantage counterpart:
df.loc[136, 'Time'] = '2:05:44' # was 2:05:38
df.loc[1527, 'Time'] = '2:13:09' # was 2:12:48
df.loc[10975, 'Time'] = '1:30:32' # was 1:30:31

## *Character* column

Again, looked through the unique values of the column.
```python
sorted(df['Character'].dropna().unique())

# >>> ['1 freq', '20 freq', 'Arkhan', 'Doty', 'Garthok', 'Gern',
#      'Gloomstalker', 'Grenade', 'Grog', 'Kashaw', 'Kerrek',
#      'Keyleth', 'Lillith', 'Lionel', 'Lyra', 'Others', 'PIke',
#      'Percy', 'Pike', 'Scanlan', 'Shale', 'Shark', 'Sprigg',
#      'Taryon', 'Thorbir', 'Tiberius', 'Tibierus', 'Tova',
#      'Trinket', "Vax'ildan", "Vex'ahlia", 'Zahra']
```

I'll remove the rows that contain '1 freq' and '20 freq' later. For now:

In [7]:
# Correct mistyped character names:
to_replace = {
  'PIke' : 'Pike',
  'Tibierus' : 'Tiberius',
}
df['Character'].replace(to_replace, inplace=True)

## *Type of Roll* column

There are a few typos (like "Steath") and inconsistencies (two names for the same thing, like "Wisdom Save" and "Wisdom Saving").

I found them by looking through unique values in the column.
```python
sorted(df['Type of Roll'].dropna().unique())

# >>> 'Acrobatics',
#     'Alchemy?',
#     'Animal Handling',
#     ...
#     'Wisdom Save',
#     'Wisdom Save?',
#     'Wisdom Saving'
```

In [8]:
# Correcting typos and inconsistent naming in Type of Roll column:
to_replace = {
  'Steath' : 'Stealth',
  'Dex Save' : 'Dexterity Save',
  'Wisdom Saving' : 'Wisdom Save',
  'Beard' : 'Beard Check',
  'Ressurrection Roll' : 'Resurrection Roll',
  'Fix' : 'Tinkering',
  'Spell Effect' : 'Intelligence',
  'Skill' : 'Constitution Save',
}
df['Type of Roll'].replace(to_replace, inplace=True)

## *Total Value* column

I've looked through the unique values in the column.

```python
sorted(df['Total Value'].dropna().astype(str).unique())

# >>> ['--', '-1', '0', '1', '10', '102', '109', '11', '12', '13', '14',
#      '15', '16', '17', '18', '19', '2', '20', '20+', '21', '22', '23',
#      ...
#      'Nat3', 'Nat4', 'Nat6', 'Nat8', 'Natural 1', 'Natural 20',
#      'Success', 'Unkknown', 'Unknown', 'Unnknown', 'unknown']
```

There are a number of issues:
1. Typos.
1. Inconsistent notation.
1. Natural value of a roll is written instead of its total.
1. Total value is missing despite natural value being present.

Correction:

In [9]:
# Replace inconsistent names and typos:
to_replace = {
  'Natural 20' : 'Nat20',
  'Natural 1' : 'Nat1',
  'Nat' : 'Nat1',
  '20+' : '>20',
  '70ish' : '>70',
  'Unkknown' : 'Unknown',
  'Unnknown' : 'Unknown',
  'unknown' : 'Unknown',
}
df['Total Value'].replace(to_replace, inplace=True)

# Remove Nat20= to leave pure values:
mask = df['Total Value'].str.contains('Nat20=', na=False)
df.loc[mask, 'Total Value'] = \
  df.loc[mask, 'Total Value'].str.replace('Nat20=', '')

# Remove Nat1= to leave pure values:
mask = df['Total Value'].str.contains('Nat1=', na=False)
df.loc[mask, 'Total Value'] = \
  df.loc[mask, 'Total Value'].str.replace('Nat1=', '')

# Calculate missing total values from natural values:
df.loc[213, 'Total Value'] = '8' # was Nat2
df.loc[251, 'Total Value'] = '18' # was Nat16
df.loc[252, 'Total Value'] = '15' # was Nat13
df.loc[746, 'Total Value'] = '10' # was Nat3
df.loc[776, 'Total Value'] = '6' # was Nat2
df.loc[811, 'Total Value'] = '13' # was Nat2
df.loc[1162, 'Total Value'] = '13' # was Nat2
df.loc[1173, 'Total Value'] = '17' # was Nat4
df.loc[1393, 'Total Value'] = '31' # was Nat19
df.loc[1420, 'Total Value'] = '29' # was Nat17
df.loc[1446, 'Total Value'] = '9' # was --
df.loc[2216, 'Total Value'] = '1' # was --
df.loc[2331, 'Total Value'] = '21' # was Nat19
df.loc[2679, 'Total Value'] = '28' # was Nat17
df.loc[3045, ['Total Value', 'Natural Value']] = ('18', '13') # was Nat18
df.loc[3158, 'Total Value'] = '28' # was Nat18, 28
df.loc[3786, 'Total Value'] = '7' # was Nat3
df.loc[3854, 'Total Value'] = '12' # was Nat3
df.loc[3956, 'Total Value'] = '4' # was Nat2
df.loc[4077, 'Total Value'] = '23' # was Nat 18
df.loc[5395, 'Total Value'] = '26' # was Nat13
df.loc[9658, 'Total Value'] = '13' # was Nat3
df.loc[9659, 'Total Value'] = '23' # was Nat13
df.loc[9736, 'Total Value'] = '21' # was Nat6
df.loc[10266, 'Total Value'] = '20' # was Nat15
df.loc[11964, 'Total Value'] = '31' # was Nat19
df.loc[12506, 'Total Value'] = '10' # was Nat2
df.loc[12560, 'Total Value'] = '17' # was Nat6
df.loc[12581, 'Total Value'] = '12' # was Nat3
df.loc[12588, 'Total Value'] = '19' # was Nat8
df.loc[12662, 'Total Value'] = '33' # was Nat19
df.loc[12683, 'Total Value'] = '31' # was Nat19
df.loc[12757, 'Total Value'] = '17' # was Nat17

df.loc[4008, 'Total Value'] = '25' # was null
df.loc[4009, 'Total Value'] = '9' # was null
df.loc[4010, 'Total Value'] = '9' # was null
df.loc[4158, 'Total Value'] = '11' # was null
df.loc[5248, ['Total Value', 'Natural Value']] = ('5', '5') # was null, 0
df.loc[5433, 'Total Value'] = '26' # was null
df.loc[10726, 'Total Value'] = '23' # was null
df.loc[11023, 'Total Value'] = '22' # was null
df.loc[11024, 'Total Value'] = '24' # was null
df.loc[11025, 'Total Value'] = '28' # was null
df.loc[11170, 'Total Value'] = '18' # was null
df.loc[11267, ['Total Value', 'Natural Value']] = ('17', '7') # was null, Unknown
df.loc[11285, 'Total Value'] = '27' # was null
df.loc[11307, 'Total Value'] = '22' # was null
df.loc[11332, 'Total Value'] = '12' # was null
df.loc[11342, 'Total Value'] = '31' # was null
df.loc[[11671, 12220, 12223], 'Total Value'] = 'Nat19' # was null
df.loc[12194, 'Total Value'] = '2' # was null
df.loc[12335, 'Total Value'] = '14' # was null
df.loc[12385, 'Total Value'] = '32' # was null
df.loc[12767, ['Total Value', 'Natural Value']] = ('11', '11') # was Unknown, Unknown

## *Natural Value* column

```python
sorted(df['Natural Value'].dropna().astype(str).unique())

# >>> ['#REF!', '--', '-1', '-4', '0', '1', '10', '11', '12', '13', '14',
#      '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24',
#      '25', '26', '3', '4', '5', '6', '7', '8', '9', 'Uknown', 'Unknown',
#      'Unkown', 'unknown']
```

In [10]:
# Correcting typos and inconsistent naming in Natural Value column:
to_replace = {
  '--' : np.nan,
  'Uknown' : 'Unknown',
  'Unkown' : 'Unknown',
  'unknown' : 'Unknown',
}
df['Natural Value'].replace(to_replace, inplace=True)

df.loc[5811, 'Natural Value'] = '19' # was #REF!

## *Crit?* column

```python
sorted(df['Crit?'].dropna().astype(str).unique())

# >>> ['--', '1', 'Y']
```

'--' should be an empty cell; 1 should be Y.

In [11]:
# Correct inconsistent notation:
to_replace = {
  '1' : 'Y',
  '--' : np.nan
}
df['Crit?'].replace(to_replace, inplace=True)

## *# Kills* column

```python
sorted(df['# Kills'].dropna().astype(str).unique())

# ['0.5', '1', '12', '2', '3', '30', '4', '5', '6', '7', '?', '\xa0']
```

'?' and '\xa0' must go.

In [12]:
df.loc[[190, 3883, 3890], '# Kills'] = np.nan # were ?, \xa0, \xa0

## *Damage* and *Notes* columns

### Typos

The process here is a bit more involved, because there are a lot of unique values. To find typos, I'm comparing each word from Damage and Notes against a [list](https://github.com/dwyl/english-words) of English words.

```python
with open('drive/MyDrive/words_dictionary.json') as f:
  all_words = json.load(f)

# Augment the dictionary with Critical Role jargon, names, and abbreviations.
# 1 does not mean anything; it can be any value.
special = {'allura': 1, 'aoo': 1, 'arbuckle' : 1, 'arkhan' : 1, 'arkhans': 1,
           'avandras': 1, 'battleaxe': 1, 'behir' : 1, 'bigbys': 1, 'bloodaxe': 1,
           'bloodseeking': 1, 'bludgeoningpiercing' : 1, 'bulette' : 1,
           'cerkonos' : 1, 'clarota' : 1, 'cloaker': 1, 'critinclusive': 1,
           'daxio' : 1, 'direbear' : 1, 'direwolf' : 1, 'dragonslayer': 1,
           'dragonslaying': 1, 'duergar' : 1, 'dwarven': 1, 'efreeti' : 1,
           'elephantkeyleth' : 1, 'erinyes' : 1, 'faneeater': 1, 'fassbender' : 1,
           'fbwh': 1, 'feeblemind': 1, 'fenthras': 1, 'feymire' : 1,
           'firebludgeoning' : 1, 'fireradiant' : 1, 'flameskull' : 1,
           'flametongue': 1, 'fmc' : 1, 'fomorian': 1, 'gern': 1, 'ghurrix' : 1,
           'gilmore' : 1, 'gloomstalker' : 1, 'greatsword': 1, 'greenbeard' : 1,
           'gricks' : 1, 'groon': 1, 'gsof': 1, 'gwm': 1, 'halfdamage': 1,
           'hdywtdt': 1, 'hotis' : 1, 'illithid' : 1, 'iotha' : 1, 'kashaw': 1,
           'kaylie' : 1, 'kerrek': 1, 'kerrion' : 1, 'kevdak' : 1, 'keyleth': 1, 
           'kima': 1, 'kryyn' : 1, 'kvarn': 1, 'kynan' : 1, 'lboss': 1,
           'lemure' : 1, 'lfe' : 1, 'lifestealing': 1, 'lizardfolk' : 1,
           'longsword': 1, 'luska' : 1, 'mathmod': 1, 'metamagic': 1, 'minxy': 1,
           'murghol' : 1, 'mythcarver': 1, 'nat': 1, 'nonlethal': 1, 'nonmagic': 1,
           'nothic': 1, 'npc': 1, 'ogden' : 1, 'ogrillon' : 1, 'omg': 1,
           'orog' : 1, 'orthax' : 1, 'otyugh' : 1, 'percy': 1, 'piercinglightning' : 1,
           'planetar': 1, 'poisondagger': 1, 'potd': 1, 'pwat': 1, 'raishan' : 1,
           'rimefang' : 1, 'ripley': 1, 'ripleys': 1, 'saundor' : 1, 'scanlan': 1,
           'scanlans': 1, 'selfhealing': 1, 'silverlaced': 1, 'slashingnecrotic' : 1,
           'slashingpsychic' : 1, 'slashingpsychicradiant' : 1, 'sprigg': 1,
           'suda' : 1, 'sylas' : 1, 'symphior' : 1, 'taliesin': 1, 'tary': 1,
           'taryon': 1, 'thorbir' : 1, 'thordak' : 1, 'thunderwave': 1, 
           'tiberius': 1, 'titanstone': 1, 'tova': 1, 'treant' : 1, 'ulara' : 1,
           'umbrasyl' : 1, 'utugash' : 1, 'vax': 1, 'vaxildan': 1, 'vaxs': 1,
           'vecna' : 1, 'vecnas': 1, 'velora' : 1, 'vex': 1, 'vexahlia': 1,
           'vexs': 1, 'vorugal' : 1, 'vouk' : 1, 'warhammer': 1, 'witchbolt': 1,
           'xanthas' : 1, 'yenk' : 1, 'zd': 1
}

all_words.update(special)

def find_typos(column, all_words):
  """Returns cells from col_name that have words not in all_words."""
  for i, note in column.items():
    if pd.notnull(note):
      note = note.replace('\n', ' ')
      clean_note = ''.join(s for s in note if s.isalpha() or s == ' ' or s == '\n')
      for word in clean_note.split():
        if word.lower() not in all_words:
          print(i, note, word.lower())

find_typos(df['Damage'], all_words)
print()
find_typos(df['Notes'], all_words)

# >>> 746 Against Basilisk.Tiberius is Restrained basilisktiberius
#     2457 17 Piercing to Ettin ettin
#     4304 13 Psychic to Vax'idlan vaxidlan
#     6388 29 to GB, Archer, Duid, 6 Baddies;  14 (29/2) to Kevdak gb
#     6388 29 to GB, Archer, Duid, 6 Baddies;  14 (29/2) to Kevdak duid
#     8898 23 to Wyvern, 11 to Rder rder
#     ...
```


Introducing abbreviated monster names does not seem like a good idea. It causes two problems:
1. One entity can have two names (e.g. Smelter Guardian 1 and SG1 are both present in the table, but they are the same monster).
1. Two entities can have the same name (e.g. A1 is both Archer 1 and Assassin 1).

I decided to replace, with a few exceptions (HP, NPC, HDYWTDT), all abbreviations.

To find them I went through all short tokens (lengths 2, 3, 4):
```python
for length in [2, 3, 4]:
  lst = []
  for s in df['Damage'].astype(str):
    lst.extend(token for token in s.split() 
                     if len(token) == length
                        and not token.isnumeric()
              )
  for s in df['Notes'].astype(str):
    lst.extend(token for token in s.split() 
                     if len(token) == length
                        and not token.isnumeric()
              )
  print(sorted(set(lst)))

# >>>
# [..., '5]', '7]', '8,', '9!', '9)', '9?', '??', 'A1', 'A2', 'A3', 'AH', 'AP', 'As', 'B1', 'B2', ...]
# [..., '>8,', 'A1,', 'AE1', 'AE2', 'AH,', 'AP,', 'Air', 'All', 'AoO', 'Arm', 'Axe', 'BB)', 'BB]', ...]
# [..., 'Edge', 'Elf,', 'Eyes', 'FAIL', 'FBWH', 'FE2,', 'FG1)', 'FG2)', ...]
```

Code that makes corrections:

In [13]:
# SS can mean Small Salamander and Sharpshooter.
# Here, I replace SS here with Small Salamander; later, with Sharpshooter.
df.loc[[5470, 5493], 'Damage'].replace(regex={'SS' : 'Small Salamander'},
                                       inplace=True
                                      )

# Replace a single occurrence of FA:
df.loc[5493, 'Damage'] = \
  df.loc[5493, 'Damage'].replace('FA', 'Fatty Arbuckle')

# Targeted replacement of Cass with Cassandra, so that I don't get "Cassandraandra". 
df.loc[[3843, 3846, 4127, 4171, 4172],
       ['Damage', 'Notes']
      ].replace(
                regex={'Cass' : 'Cassandra'},
                inplace=True
               )

# "A" can mean Archer and Assassin.
# Here, I replace "A" here with Archer; later, with Assassin.
df.loc[6280:6414, ['Damage', 'Notes']] = \
  df.loc[6280:6414, ['Damage', 'Notes']].replace(regex={'A1' : 'Archer 1',
                                                        'A2' : 'Archer 2'
                                                       }, 
                                                )

# FG can mean Frost Giant and Fire Giant.
# Here, I replace FG here with Frost Giant; later, with Fire Giant.
df.loc[1943:2123, ['Damage', 'Notes']] = \
  df.loc[1943:2123, ['Damage', 'Notes']].replace(regex={'FG1' : 'Frost Giant 1',
                                                        'FG2' : 'Frost Giant 2'
                                                       }, 
                                                )

# G1 is present in G1, as Guard 1, and in SG1, as Smelter Guardian 1.
# Here, I replace G with Guard; later, I replace SG with Smelter Guardian.
df.loc[3225:3651, ['Damage', 'Notes']] = \
  df.loc[3225:3651, ['Damage', 'Notes']].replace(regex={'G1' : 'Guard 1',
                                                        'G2' : 'Guard 2',
                                                        'G3' : 'Guard 3',
                                                        'G4' : 'Guard 4',
                                                        'G5' : 'Guard 5',
                                                        'G6' : 'Guard 6',
                                                        'G8' : 'Guard 8',
                                                       },
                                                )
# Correcting typos in Damage:
damage_typos = {  
  '\xa0' : ' ',
  'Basilisk.Tiberius' : 'Basilisk. Tiberius',
  'Budgeoning' : 'Bludgeoning',
  'Cassnadra' : 'Cassandra',
  'Duid' : 'Druid',
  'Gloomtalker' : 'Gloomstalker',
  'Gog' : 'Grog',
  'LFe' : 'LFE',
  'Lightnint' : 'Lightning',
  'Rder' : 'Rider',
  'Sanlan' : 'Scanlan',
  'utugash' : 'Utugash',
  "Vax'idlan" : "Vax'ildan",
  'Vaxildan' : "Vax'ildan",
}

df['Damage'].replace(regex=damage_typos, inplace=True)

# Correcting typos in Notes:
notes_typos = {
  '-HDYWTDT' : ' - HDYWTDT',
  "Arkan's" : "Arkhan's",
  'Blde' : 'Blade',
  'Celing' : 'Ceiling',
  'constituion' : 'constitution',
  'Dimunition' : 'Diminution',
  'disadvantaqge' : 'disadvantage',
  'disdavantage' : 'disadvantage',
  'disdvantage' : 'disadvantage',
  'ddvantage' : 'advantage',
  'Ensare' : 'Ensnare',
  'Envervate' : 'Enervate',
  'HDYTWDT' : 'HDYWTDT',
  'healls' : 'heals',
  'Mindcontrolled' : 'Mind-controlled',
  'Opportunity-Punch' : 'Opportunity - Punch',
  'Possesssed' : 'Possessed',
  'posessed' : 'possessed',
  'proficent' : 'proficient',
  'PWAT' : 'PWaT',
  'PWT' : 'PWaT',
  'Reckles ' : 'Reckless',
  'Rckless' : 'Reckless',
  'Recklesss' : 'Reckless',
  'Spritiual' : 'Spiritual',
  'Spritual' : 'Spiritual',
  'suceeds' : 'succeeds',
  'Suprise' : 'Surprise',
  'Surpise' : 'Surprise',
  'Throns' : 'Thorns',
  'telekineis' : 'telekinesis',
  'Titanstonke' : 'Titanstone',
  'toadvantage' : 'to advantage',
  'Wihout' : 'Without',
  'Wirh' : 'With',
  'Wounds-Scanlan' : 'Wounds - Scanlan',
  "Wounds-Vax'ildan" : "Wounds - Vax'ildan",
  'Unnanounced' : 'Unannounced',
  'unnanounced' : 'unannounced',
}

df['Notes'].replace(regex=notes_typos, inplace=True)

# Replace abbreviations
abbreviations = {
  '2H' : '2-handed',
  'AoO' : 'Attack of Opportunity',
  'A1' : 'Assassin 1',
  'A2' : 'Assassin 2',
  'A3' : 'Assassin 3',
  'AE' : 'Air Elemental',
  'AH' : 'Adamantine Hammer',
  'AP' : 'Archpriest',
  'B1' : 'Basilisk 1',
  'B2' : 'Basilisk 2',
  'B3' : 'Basilisk 3',
  'B4' : 'Basilisk 4',
  'B5' : 'Basilisk 5',
  'BB' : 'Blazing Bowstring',
  'BN' : 'Bad News',
  'CE' : 'Craven Edge',
  'CG' : 'Cobalt Golem',
  'DoV' : 'Dagger of Venom',
  'DS' : 'Divine Smite',
  'FBWH' : 'Firebrand Warhammer',
  'EE' : 'Earth Elemental',
  'FE1' : 'Fire Elemental 1',
  'FE2' : 'Fire Elemental 2',
  'FE3' : 'Fire Elemental 3',
  'FG1' : 'Fire Giant 1',
  'FG2' : 'Fire Giant 2',
  'FG' : 'Fire Giant ',
  'FMC' : 'Feymire Crocodile',
  'FS' : 'Flameskull',
  'GB' : 'Greenbeard',
  'GSoF' : 'Greatsword of Frenzy',
  'GWM' : 'Great Weapon Master',
  'HM' : "Hunter's Mark",
  'KT' : 'Kuo-toa ',
  'L1' : 'Looter 1',
  'L2' : 'Looter 2',
  'L3' : 'Looter 3',
  'L4' : 'Looter 4',
  'L5' : 'Looter 5',
  'L6' : 'Looter 6',
  'L7' : 'Looter 7',
  'L8' : 'Looter 8',
  'L9' : 'Looter 9',
  'LBoSS' : 'Longbow of Sky Sentinel',
  'LFE' : 'Large Fire Elemental',
  'LS' : 'Large Salamander',
  'PotD' : 'Plate of the Dawnmartyr',
  'PWaT' : 'Pass Without a Trace',
  'R1' : 'Rider 1',
  'R2' : 'Rider 2',
  'R3' : 'Rider 3',
  'R4' : 'Rider 4',
  'R5' : 'Rider 5',
  'R6' : 'Rider 6',
  'R7' : 'Rider 7',
  'R8' : 'Rider 8',
  'RT' : 'Reliable Talent',
  'SA' : 'Sneak Attack',
  'SFE' : 'Small Fire Elemental',
  'SG1' : 'Smelter Guardian 1',
  'SG2' : 'Smelter Guardian 2',
  'SG3' : 'Smelter Guardian 3',
  'SG' : 'Smelter Guardian',
  'SM' : 'Stitch Monster',
  'SS' : 'Sharpshooter',
  'T1' : 'Troll 1',
  'T2' : 'Troll 2',
  'T3' : 'Troll 3',
  'VM' : 'Vox Machina',
  'VS' : 'Vampire Spawn',
  'W1' : 'Wyvern 1',
  'W2' : 'Wyvern 2',
  'W3' : 'Wyvern 3',
  'W4' : 'Wyvern 4', 
  'WM2' : 'War Monger 2',
  'WM3' : 'War Monger 3',
  'w/' : 'with',
  'ZD' : 'Zombie Dwarf'
}

# df['Damage'].replace(regex=abbreviations, inplace=True)
# df['Notes'].replace(regex=abbreviations, inplace=True)

### Cleaning up advantages and disadvantages in Notes

I have noticed several problems regarding rolls with advantage or disadvantage:
1. Sometimes there is a mismatch between two rolls from a pair: according to Notes, one roll is at advantage and the other is at disadvantage.
2. Sometimes only one of rolls from a pair is marked in Notes. I call them _orphans_.

The code below finds the mismatches.

In [14]:
def find_orphan(mask):
  """Returns positions of orphans given a mask.
     An orphan is a **single** True surrounded by two Falses.
  """

  mask_1_forward = np.roll(mask.to_numpy(), 1)
  mask_1_backwards = np.roll(mask.to_numpy(), -1)
  has_neighbors_mask = mask & mask_1_forward | mask & mask_1_backwards
  is_orphan_mask = mask & ~has_neighbors_mask
  return np.where(is_orphan_mask)[0]

```python
disadv_mask = (df['Notes'].str.contains('disadvantage').fillna(False)
             | df['Notes'].str.contains('Disadvantage').fillna(False)
)
adv_mask = ((df['Notes'].str.contains('advantage').fillna(False)
             | df['Notes'].str.contains('Advantage').fillna(False)
            )
            & ~disadv_mask
)

adv_orphan = find_orphan(adv_mask)
disadv_orphan = find_orphan(disadv_mask)

# Find rows where orphan advantage and orphan disadvantage are neighbors:
adv_disadv_mismatch = np.sort(np.concatenate((adv_orphan, disadv_orphan)))
idx = adv_disadv_mismatch[adv_disadv_mismatch == np.roll(adv_disadv_mismatch-1, -1)]

# Print mismatching rolls and their neighbors:
print(df[['Episode',
          'Time',
          'Character',
          'Type of Roll',
          'Natural Value',
          'Notes'
         ]
        ].iloc[np.sort(np.concatenate((idx-1, idx, idx+1)))])

# >>>
# 300         4  1:34:00  Vax'ildan         Acrobatics             5                                         NaN
# 301         4  1:36:40    Scanlan            Stealth       Unknown             Disregarded due to disadvantage
# 302         4  1:36:40    Scanlan            Stealth            11                              With advantage
# 782         7  3:09:26   Tiberius  Constitution Save           NaN                                 Unknown mod
# 783         7  3:09:32      Percy             Attack       Unknown                Disregarded due to advantage
# 784         7  3:09:32      Percy             Attack             5                 Pepperbox with Disadvantage
# ...
# 13116     113  4:17:09       Pike          Dexterity             4                                         NaN
# 13117     113  4:19:25    Scanlan      Charisma Save       Unknown             Disregarded due to disadvantage
# 13118     113  4:19:25    Scanlan      Charisma Save             8                   Scanlan 1, With advantage
# 13339     114  3:26:18  Vax'ildan             Damage           NaN                                         NaN
# 13340     114  3:27:17  Vax'ildan             Attack       Unknown  Disregarded due to disadvantage with Bless
# 13341     114  3:27:17  Vax'ildan             Attack       Unknown           Whisper with advantage with Bless
```

In [15]:
# Correcting mismatching advantage and disadvantage for a roll:

to_adv_list = [301, 2987, 2989, 2991, 7327, 12146, 13117, 13340]
df.loc[to_adv_list, 'Notes'] = \
  df.loc[to_adv_list, 'Notes'].replace(regex={'disadvantage' : 'advantage'})                                 
                                     
to_disadv_list = [783, 857, 1466, 2625, 3629, 4483, 4487]
df.loc[to_disadv_list, 'Notes'] = \
  df.loc[to_disadv_list, 'Notes'].replace(regex={'advantage' : 'disadvantage'})

#### Adding missing remarks on having advantage

For some reason, almost always it is _With advantage_ remark that is missing, while _Disregarded due to advantage_ is present.

Whenever a note mentions advantage but either no rolls were made or only one was made, I shorten advantage to disadv. (Because this is the same as a straight roll, and, for my own project, I need a way to filter rolls at advantage and disadvatage.)

In [16]:
# Special cases with no pattern:
df.loc[9478:9532:2, 'Notes'] += ' (with advantage)'
df.loc[[1531, 3343, 6305, 8149, 8151, 11854], 'Notes'] += ' (with advantage)'
df.loc[[5104, 8300, 11183, 11383, 11402, 13209], 'Notes'] += 'and advantage'

df.loc[1527, 'Notes'] = ''
df.loc[3212, 'Notes'] = 'With advantage, disregarded due to Luck'
df.loc[3393, 'Notes'] = 'Pepperbox Sharpshooter Ice (adv. not rolled)'
df.loc[4743, 'Notes'] = 'Craven Edge GWM Reckless Attack (with advantage) - Frost Worm dies - HDYWTDT'
df.loc[[4918, 4945, 4947], 'Notes'] = 'Craven Edge Great Weapon Master Reckless Attack (with advantage), Grog gains +1 Strength'
df.loc[7195, 'Notes'] = "Bad News Deadeye Sharpshooter (adv. not rolled)"
df.loc[8153, 'Notes'] = "Bloodaxe Great Weapon Master Reckless Attack (with advantage), slices through Ripley's midsection"
df.loc[9835, 'Notes'] = 'With Luck'
df.loc[10698, 'Notes'] = 'Against Cage drop (with advantage), Grog is restrained'
df.loc[11393, 'Notes'] = 'With Pass Without a Trace and advantage, bad math/mod or unannounced nat20'
df.loc[11943, 'Notes'] = 'Adv. not rolled'
df.loc[13320, 'Notes'] = 'Whisper Sneak Attack of Opportunity with Divine Smite, Bless, and advantage'
df.loc[13447, 'Notes'] = 'Sword of Kas Reckless'

In [17]:
disadv_mask = (df['Notes'].str.contains('disadvantage').fillna(False)
             | df['Notes'].str.contains('Disadvantage').fillna(False)
)
adv_mask = ((df['Notes'].str.contains('advantage').fillna(False)
             | df['Notes'].str.contains('Advantage').fillna(False)
            )
            & ~disadv_mask
)

adv_orphan = find_orphan(adv_mask)
disadv_orphan = find_orphan(disadv_mask)

# Special cases; not orphans
not_orphan_idx = np.where(np.isin(adv_orphan, 
                                  [202, 204, 3343, 3346, 11170, 13289, 13291]))
adv_orphan = np.delete(adv_orphan, not_orphan_idx)

# Among orphaned advantages:
for i in adv_orphan:
  # If the previous line has matching Time, Character, Type of Roll:
  if (df.loc[i, ['Time', 'Character', 'Type of Roll']] 
      == df.loc[i-1, ['Time', 'Character', 'Type of Roll']]
     ).all():
    # If the cell is empty, fill it:
    if pd.isna(df.loc[i-1, 'Notes']):
      df.loc[i-1, 'Notes'] = 'With advantage'
    # Otherwise, add ' (with disadvantage)' to the note:
    else:
      df.loc[i-1, 'Notes'] += ' (with advantage)'

  # If the next line has matching Time, Character, Type of Roll:
  if (df.loc[i, ['Time', 'Character', 'Type of Roll']] 
      == df.loc[i+1, ['Time', 'Character', 'Type of Roll']]
     ).all():
    # If the cell is empty, fill it:
    if pd.isna(df.loc[i+1, 'Notes']):
      df.loc[i+1, 'Notes'] = 'With advantage'
    # Otherwise, add ' (with disadvantage)' to the note:  
    else:
      df.loc[i+1, 'Notes'] += ' (with advantage)'

#### Filling missing remarks on disadvantage

For some reason, almost always it is _With disadvantage_ remark that is missing, while _Disregarded due to disadvantage_ is present.

Whenever a note mentions disadvantage but either no rolls were made or only one was made, I shorten disadvantage to disadv. (Because this is the same as a straight roll, and, for my own project, I need a way to filter rolls at advantage and disadvatage.)

In [18]:
# Special cases with no pattern:
df.loc[[854, 860, 1110, 1467, 1469], 'Notes'] += ' with disadvantage'
df.loc[[5752, 5754, 5756], 'Notes'] += ' (was rolled with disadvantage)'
df.loc[10256, 'Notes'] = 'Whisper with disadvantage and Bless'

df.loc[2785, 'Notes'] = 'Sylas has disadv. on all attacks'
df.loc[3095, 'Notes'] = 'Behir has disadv.'
df.loc[3777, 'Notes'] = 'Has disadv. on Dexterity Ability Checks'
df.loc[5767, 'Notes'] = 'Arrow, No disadv. due to Guiding Bolt'
df.loc[6379, 'Notes'] = 'Disadv. not rolled'
df.loc[6935, 'Notes'] = 'Longbow of Sky Sentinel (disadv. not rolled)'
df.loc[4321, 'Notes'] = 'Cutting Words, has disadv. on next attack'
df.loc[9993, 'Notes'] = 'Dagger of Venom (disadv. not rolled)'

In [19]:
disadv_mask = (df['Notes'].str.contains('disadvantage').fillna(False)
             | df['Notes'].str.contains('Disadvantage').fillna(False)
)
disadv_orphan = find_orphan(disadv_mask)

# Special cases; not orphans:
not_orphan_idx = np.where(np.isin(disadv_orphan, [601, 605, 6608, 10179, 10181, 10251, 10253, 13244, 13246])) # special cases; not orphans
disadv_orphan = np.delete(disadv_orphan, not_orphan_idx)

# Among orphaned disadvantages:
for i in disadv_orphan:
  # If the previous line has matching Time, Character, Type of Roll:
  if (df.loc[i, ['Time', 'Character', 'Type of Roll']] 
      == df.loc[i-1, ['Time', 'Character', 'Type of Roll']]
     ).all():
    # If the cell is empty, fill it:
    if pd.isna(df.loc[i-1, 'Notes']):
      df.loc[i-1, 'Notes'] = 'With disadvantage'
    else:
      df.loc[i-1, 'Notes'] += ' (with disadvantage)'

  # If the next line has matching Time, Character, Type of Roll:
  if (df.loc[i, ['Time', 'Character', 'Type of Roll']] 
      == df.loc[i+1, ['Time', 'Character', 'Type of Roll']]
     ).all():
    # If the cell is empty, fill it:
    if pd.isna(df.loc[i+1, 'Notes']):
      df.loc[i+1, 'Notes'] = 'With disadvantage'
    # Otherwise, add ' (with disadvantage)' to the note:
    else:
      df.loc[i+1, 'Notes'] += ' (with disadvantage)'

## Insert rows

In [20]:
# Add a row for a missing advantage counterpart:
line = pd.DataFrame({
    'Episode': 'C1E004',
    'Time' : '1:46:22',
    'Character' : "Vax'ildan",
    'Type of Roll' : 'Attack',
    'Total Value' : 'Unknown',
    'Natural Value' : 'Unknown',
    'Crit?' : '',
    'Damage' : '',
    '# Kills' : '',
    'Notes' : 'Disregarded due to advantage',
    }, index=[315.5]
)
df = df.append(line, ignore_index=False)

line = pd.DataFrame({
    'Episode': 'C1E108',
    'Time' : '0:47:35',
    'Character' : "Grog",
    'Type of Roll' : 'Attack',
    'Total Value' : '25',
    'Natural Value' : '14',
    'Crit?' : '',
    'Damage' : '',
    '# Kills' : '',
    'Notes' : 'Disregarded due to advantage',
    }, index=[12117.5]
)
df = df.append(line, ignore_index=False)

## Delete rows

In [21]:
# Row 1838 is a dexterity save for monsters, rolled by the DM:
df.drop(1838, inplace=True)
# Row 4129 is a duplicate of row 4135:
df.drop(4129, inplace=True)
# Drop empty rows in the middle of the table:
df.drop(np.arange(7244, 7249), inplace=True)

## Miscellaneous corrections

Found randomly.

In [22]:
# AoE damage from 2 Fireballs, brought where they belong to:
df.loc[1829, 'Damage'] = ('41 Fire to Hydra, 22 (41/2) Fire to Grog, '
                          + '22 (41/2) Fire to Trinket, 41 Fire to 4 Others, '
                          + '22 (41/2) Fire to 1 Other'
)
df.loc[1830, 'Damage'] = ("39 Fire to Hydra, 18 (39/2) Fire to Vax'ildan, "
                          + '39 Fire to Grog, 39 Fire to Trinket, '
                          + '39 Fire to 5 Others'
)
df.loc[1831:1837] = np.nan

# Was Arcana for some reason:
df.loc[5433, 'Type of Roll'] = 'Alchemy'
# Found a couple of missing critical hits:
df.loc[[12431, 12434], 'Crit ?'] = 'Y'

## Sort by *Episode*, *Time*

In [23]:
df.sort_values(['Episode', 'Time'], inplace=True)

## Mild to strict

In [25]:
df_mild = df.reset_index(drop=True)

# Expand abbreviations (abbreviations are defined in the *Damage and Notes* section):
df['Damage'].replace(regex=abbreviations, inplace=True)
df['Notes'].replace(regex=abbreviations, inplace=True)

# Rows 3032, 3324 are notes on a missing audio/video:
df.drop([3032, 3324], inplace=True)

# Replace \n with whitespace:
df.loc[:, ['Damage', 'Notes']] = \
  df.loc[:, ['Damage', 'Notes']].replace(regex={'\n' : ' '})

df.reset_index(drop=True, inplace=True)

df_mild.to_csv('Mildly_Cleanish_All_Rolls_TalDorei.csv', index=False)  
df.to_csv('Strictly_Cleanish_All_Rolls_TalDorei.csv', index=False)